### Imports

In [2]:
# pandas and numpy
import pandas as pd
import numpy as np

# nltk imports
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer

# sci-kit learn imports
from sklearn.compose import make_column_transformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
from sklearn.pipeline import Pipeline, make_pipeline

# Presentation and visuals
import seaborn as sns
import matplotlib.pyplot as plt

# This magic line will allow you to generate plots
# within the Jupyter notebook.
%matplotlib inline
from pprint import pprint
pd.options.display.max_seq_items = 2000
pd.options.display.max_rows = 4000
# pd.set_option(display.max_columns), None

# other imports
import json
import lxml
from lxml import html
import random
import regex as re
import requests
import time
import urllib.request
from datetime import datetime



In [4]:
df = pd.read_csv('../data/processed/canadian.csv')

In [7]:
df.tail()


,title,authors,image,description
7999,Zero Day,Ezekiel Boone,https://images.49thshelf.com/var/ezflow_site/s...,"The wildly entertaining, deeply satisfying fin..."
8000,Zip's File,Shannon Maguire,https://images.49thshelf.com/var/ezflow_site/s...,Zip's File: A Romance of Silence explores the ...
8001,Zolitude,Paige Cooper,https://images.49thshelf.com/var/ezflow_site/s...,WINNER OF THE 2018 QUEBEC WRITERS' FEDERATION ...
8002,Zoo and Crowbar,David Zieroth,https://images.49thshelf.com/var/ezflow_site/s...,The Wind has mysteriously caused the death of ...
8003,"Zora, A Cruel Tale",Philippe Arseneault,https://images.49thshelf.com/var/ezflow_site/s...,Arsenault’s Rabelaisian fantasy is a gothic ta...


In [ ]:
# isbn = pd.read_csv('../data/isbnlist.csv', 'isbn' : str)
# base_url = 'https://www.biblioshare.ca/BNCServices/BNCServices.asmx/ONIX?Token=zvo3vpz7uulcuajs&EAN='
# for isbn in isbn['isbn']:
#     target_url = base_url + isbn

### Preprocessing


In [ ]:
# Preprocess the posting content; this should take under 2 minutes
# start an empty list to hold preprocessed postings

for i in range(len(df)):  # for each description
    desc = re.sub('[^a-zA-Z]', ' ', df['description'][i]) # remove non text characters
    desc = desc.lower() # lower-case everything
    desc = desc.split() # split into words
    desc = ' '.join(desc) # reassemble the string
    df['description'][i] = desc


## Modeling

### TF-IDF

In [ ]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['description'])
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

### Recommender

In [ ]:
results = {}

for idx, row in df.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], df['id'][i]) for i in similar_indices]
    results[row['id']] = similar_items[1:]

In [ ]:
def item(id):
    return df.loc[df['id'] == id]['description'].tolist()[0].split(' - ')[0]

In [ ]:
def recommend(item_id, num):
    print("Here " + str(num) + " books that are similar to " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")

In [ ]:
print('How many new books would you like to learn about?')
num_books = input()

In [ ]:
recommend(item_id=109, num=3)

### Visualization

## References

To come